##Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##import

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchvision import models
from torch.utils.data import DataLoader
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
import cv2

## Visual and Augment

In [ ]:
# Define members (subdirectories)
members = ["Oak", "Pat", "Pookkie", "Praewa", "Tup"]
image_dir = "/content/drive/MyDrive/DP/Train_Splitted/train"
augmented_dir = "/content/drive/MyDrive/DP/Augmented_Train"  # Save augmented images

def augment_pipeline(image_dir, augmented_dir, members, augmentation_factor=3, img_size=(256, 256)):
    """Loads images, applies augmentation, and saves them in `augmented_dir`."""

    os.makedirs(augmented_dir, exist_ok=True)

    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    for member in members:
        input_folder = os.path.join(image_dir, member)
        output_folder = os.path.join(augmented_dir, member)
        os.makedirs(output_folder, exist_ok=True)

        image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        print(f"Processing {member}: Found {len(image_files)} images")

        for filename in image_files:
            img_path = os.path.join(input_folder, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
            img = cv2.resize(img, img_size)
            img = np.expand_dims(img, axis=0)

            i = 0
            for batch in datagen.flow(img, batch_size=1):
                save_path = os.path.join(output_folder, f"aug_{i}_{filename}")
                cv2.imwrite(save_path, batch[0].astype(np.uint8))
                i += 1
                if i >= augmentation_factor:
                    break

        print(f"{member}: {len(os.listdir(output_folder))} images after augmentation")

# Apply augmentation and save images
augment_pipeline(image_dir, augmented_dir, members, augmentation_factor=3, img_size=(256, 256))

##Traning

In [ ]:
# Parameters
num_classes = 5  # Number of classes
batch_size = 32
epochs = 4
learning_rate = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data Preprocessing
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Ensure 3 channels
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load Dataset (Fix: Use correct directory path)
train_dataset = datasets.ImageFolder(root='/content/drive/MyDrive/DP/Augmented_Train', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.ImageFolder(root='/content/drive/MyDrive/DP/Train_Splitted/test', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Load Pretrained ResNet Model
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)  # Modify the fully connected layer
model = model.to(device)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training Loop
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate training accuracy
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    train_acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}, Training Accuracy: {train_acc:.2f}%")

##Test accuracy

In [ ]:
# Evaluation Loop (Test Accuracy)
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

test_acc = 100 * correct / total

##Save Model

In [ ]:
# Save Model
torch.save(model.state_dict(), "handwriting_resnet.pth")
print("Model saved!")

#Test prediction

In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image
from torchvision import models

# Load the saved model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet18(pretrained=False)  # Don't load pretrained weights
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 5)  # Match the number of classes
model.load_state_dict(torch.load("handwriting_resnet.pth", map_location=device))
model = model.to(device)
model.eval()

# Define the image transformations
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Ensure 3 channels
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load and preprocess the image
image_path = "path/to/your/image.jpg"  # Replace with the actual path to your image
image = Image.open(image_path)
image = transform(image).unsqueeze(0).to(device)

# Make the prediction
with torch.no_grad():
  output = model(image)
  _, predicted_class = torch.max(output, 1)

# Print the predicted class
print(f"Predicted class: {predicted_class.item()}")

# Define class names (adjust to your actual classes)
class_names = ["Oak", "Pat", "Pookkie", "Praewa", "Tup"]

# Print the predicted class name
predicted_class_name = class_names[predicted_class.item()]
print(f"Predicted class name: {predicted_class_name}")
